In [3]:
f = open("memory_profiler.log", "r")
lines = [line.replace('\x00','').replace('\n','') for line in f.readlines()]
for i,line in enumerate(lines):
    if line.find('FORWARD PASS START') !=-1: start_index = i
    if line.find('FORWARD PASS END') !=-1: end_index = i
lines = lines[start_index+1:end_index]

In [4]:
memory_list = []
for line in lines:
    line = line[line.find('MiB')+3:]
    line = line[:line.find('MiB')]
    memory_list.append(float(line.strip()))
print(memory_list)

[196.9, 196.3, 196.1, 147.2, 245.5, 49.0, 245.0, 49.0, 148.1, 24.5, 123.1, 24.5, 74.6, 12.3, 61.3, 12.3, 37.8, 6.1, 30.6, 6.1, 0.0]


In [5]:
layer_list = []
for line in lines:
    line = line[line.find('MiB')+3:]
    layer_list.append(line.split('=')[1])
print(layer_list)

[' self.conv1(x)', ' self.bn1(out)', ' self.relu1(out)', ' self.mp1(out)', ' self.bn3(self.conv3(self.relu2(self.bn2(self.conv2(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line', ' self.relu2(out)', ' self.bn5(self.conv5(self.relu3(self.bn4(self.conv4(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line', ' self.relu3(out)', ' self.bn7(self.conv7(self.relu4(self.bn6(self.conv6(out))))) + self.downsample20(out) # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line', ' self.relu4(out)', ' self.bn9(self.conv9(self.relu5(self.bn8(self.conv8(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression 

### Note: The output for the above two cells for your Neural Network should be as shown below
### print(memory_list)
\[49.9, 49.4, 0.1, 49.2, 49.0, 49.0, 98.2, 98.0, 98.0, 24.6, 24.5, 24.5, 49.2, 49.0, 49.0, 49.1, 49.0, 49.0, 49.3, 49.0, 49.0, 12.4, 12.3, 12.3, 24.8, 24.5, 24.5, 24.6, 24.5, 24.5, 24.9, 24.5, 24.5, 6.2, 6.1, 6.1, 12.8, 12.3, 12.3, 12.4, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 3.1, 3.1, 3.1, 6.3, 6.1, 6.1, 6.2, 6.1, 6.1, 6.2, 6.1, 6.1, 0.1\]
### print(layer_list)
\[' self.conv1(a)', ' self.b1(out)', ' self.relu1(out)', ' self.conv20(out)', ' self.b20(out)', ' self.relu20(out)', ' self.conv21(out)', ' self.b21(out)', ' self.relu21(out)', ' self.conv30(out)', ' self.b30(out)', ' self.relu30(out)', ' self.conv31(out)', ' self.b31(out)', ' self.relu31(out)', ' self.conv40(out)', ' self.b40(out)', ' self.relu40(out)', ' self.conv41(out)', ' self.b41(out)', ' self.relu41(out)', ' self.conv50(out)', ' self.b50(out)', ' self.relu50(out)', ' self.conv51(out)', ' self.b51(out)', ' self.relu51(out)', ' self.conv60(out)', ' self.b60(out)', ' self.relu60(out)', ' self.conv61(out)', ' self.b61(out)', ' self.relu61(out)', ' self.conv70(out)', ' self.b70(out)', ' self.relu70(out)', ' self.conv71(out)', ' self.b71(out)', ' self.relu71(out)', ' self.conv80(out)', ' self.b80(out)', ' self.relu80(out)', ' self.conv81(out)', ' self.b81(out)', ' self.relu81(out)', ' self.conv90(out)', ' self.b90(out)', ' self.relu90(out)', ' self.conv91(out)', ' self.b91(out)', ' self.relu91(out)', ' self.conv100(out)', ' self.b100(out)', ' self.relu100(out)', ' self.conv101(out)', ' self.b101(out)', ' self.relu101(out)', ' self.conv110(out)', ' self.b110(out)', ' self.relu110(out)', ' self.conv111(out)', ' self.b111(out)', ' self.relu111(out)', ' self.conv120(out)', ' self.b120(out)', ' self.relu120(out)', ' self.conv121(out)', ' self.b121(out)', ' self.relu121(out)', ' self.conv130(out)', ' self.b130(out)', ' self.relu130(out)', ' self.conv131(out)', ' self.b131(out)', ' self.relu131(out)', ' self.conv140(out)', ' self.b140(out)', ' self.relu140(out)', ' self.conv141(out)', ' self.b141(out)', ' self.relu141(out)', ' self.avgpl(out)'\]
### If the file was not parsed properly or the output is not as expected please compile the memory_list and layer_list manually using the memory_profiler provided in Step1 or using any other technique.

In [6]:
### Algorithm to calculate the optimal Checkpointing Configuration using empirical method based on the memory profiled in previous step.

def check(mid, array, n, K): 
    count = 0
    sum = 0
    initial_pass_memory = 0
    checkpoint_indices = []
    for i in range(n): 
          
        # If individual element is greater maximum possible sum 
        if (array[i] > mid): 
            return False, initial_pass_memory, checkpoint_indices
  
        # Increase sum of current sub - array 
        sum += array[i] 
  
        # If the sum is greater than mid increase count 
        if (sum > mid): 
            count += 1
            checkpoint_indices.append(i)
            sum = 0
            initial_pass_memory += array[i]
    count += 1
  
    # Check condition 
    if (count <= K): return True, initial_pass_memory, checkpoint_indices
    return False, initial_pass_memory, checkpoint_indices
  
# Function to find maximum subarray sum which is minimum 
def solve(array, n, K): 
    start = 1
    end = 0
  
    for i in range(n): 
        end += array[i] 
  
    # Answer stores possible maximum sub array sum 
    answer = 0
    optimal_initial_pass_memory = 0
    optimal_checkpoint_indices = []
    while (start <= end): 
        mid = (start + end) // 2
  
        # If mid is possible solution Put answer = mid; 
        boolean, initial_pass_memory, checkpoint_indices = check(mid, array, n, K)
        if boolean: 
            answer = mid 
            optimal_initial_pass_memory = initial_pass_memory
            optimal_checkpoint_indices = checkpoint_indices
            end = mid - 1
        else: 
            start = mid + 1
  
    return answer, optimal_initial_pass_memory, checkpoint_indices

def get_optimal_checkpoint_config(memory_list, verbose=False):

    array = memory_list
    n = len(array)
    min_memory = -1
    min_config = None

    ### NOTE: n_checkpoints is the maximum no. of checkpoints that can be used
    for n_checkpoints in range(1,n):
        max_segment_memory, optimal_initial_pass_memory, optimal_checkpoint_indices = solve(array, n, n_checkpoints)
        total_memory = max_segment_memory + optimal_initial_pass_memory
        if (total_memory==0) or (max_segment_memory==0):continue
        if (min_memory==-1) or (total_memory<min_memory):
            min_memory=total_memory
            min_config = {  'max_segment_memory':max_segment_memory,
                            'optimal_initial_pass_memory':optimal_initial_pass_memory,
                            'total_memory':total_memory,
                            'n_checkpoints':n_checkpoints,
                            'optimal_checkpoint_indices':optimal_checkpoint_indices }

        if verbose:
            print('\nn_checkpoints =',n_checkpoints)
            print('max_segment_memory + optimal_initial_pass_memory =',round(max_segment_memory,1),'+',round(optimal_initial_pass_memory,1),'=',round(max_segment_memory+optimal_initial_pass_memory,1))
            print('Checkpoint indices=',optimal_checkpoint_indices)

    return min_config

min_config = get_optimal_checkpoint_config(memory_list, verbose=True)


n_checkpoints = 2
max_segment_memory + optimal_initial_pass_memory = 905.0 + 245.5 = 1150.5
Checkpoint indices= [4, 19]

n_checkpoints = 3
max_segment_memory + optimal_initial_pass_memory = 590.0 + 295.3 = 885.3
Checkpoint indices= [3, 8]

n_checkpoints = 4
max_segment_memory + optimal_initial_pass_memory = 414.0 + 393.2 = 807.2
Checkpoint indices= [2, 5, 8]

n_checkpoints = 5
max_segment_memory + optimal_initial_pass_memory = 295.0 + 711.6 = 1006.6
Checkpoint indices= [1, 3, 6, 10]

n_checkpoints = 6
max_segment_memory + optimal_initial_pass_memory = 266.0 + 564.6 = 830.6
Checkpoint indices= [1, 3, 5, 7, 10]

n_checkpoints = 7
max_segment_memory + optimal_initial_pass_memory = 246.0 + 595.2 = 841.2
Checkpoint indices= [1, 3, 5, 7, 10, 18]

n_checkpoints = 8
max_segment_memory + optimal_initial_pass_memory = 246.0 + 595.2 = 841.2
Checkpoint indices= [1, 3, 5, 7, 10, 18]

n_checkpoints = 9
max_segment_memory + optimal_initial_pass_memory = 246.0 + 595.2 = 841.2
Checkpoint indices= [1, 

In [7]:
print('Checkpointing Configuration:')
print(min_config)

Checkpointing Configuration:
{'max_segment_memory': 414.0, 'optimal_initial_pass_memory': 393.2, 'total_memory': 807.2, 'n_checkpoints': 4, 'optimal_checkpoint_indices': [2, 5, 8]}


In [9]:
# Print the Layers in each segment for the optimal checkpointing configuration
checkpoints = min_config['optimal_checkpoint_indices']
# checkpoints = [2, 5, 8] # Enter manually here for any other configuration corresponding to different n_checkpoints

start_index = 0
for i, checkpoint in enumerate(checkpoints):
    end_index = checkpoint
    print('Segment:',i+1)
    for layer, memory in zip(layer_list[start_index:end_index],memory_list[start_index:end_index]):
        print('Layer =',layer,', Memory =',memory)
    print('\n')
    start_index = end_index

# Last Segment
print('Segment:',len(checkpoints)+1)
for layer, memory in zip(layer_list[start_index:],memory_list[start_index:]):
    print('Layer =',layer,', Memory =',memory)
print('\n')

Segment: 1
Layer =  self.conv1(x) , Memory = 196.9
Layer =  self.bn1(out) , Memory = 196.3


Segment: 2
Layer =  self.relu1(out) , Memory = 196.1
Layer =  self.mp1(out) , Memory = 147.2
Layer =  self.bn3(self.conv3(self.relu2(self.bn2(self.conv2(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line , Memory = 245.5


Segment: 3
Layer =  self.relu2(out) , Memory = 49.0
Layer =  self.bn5(self.conv5(self.relu3(self.bn4(self.conv4(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line , Memory = 245.0
Layer =  self.relu3(out) , Memory = 49.0


Segment: 4
Layer =  self.bn7(self.conv7(self.relu4(self.bn6(self.conv6(out))))) + self.downsample20(out) # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a

In [ ]:
# NOTE: Distribute the layers of the Neural Network as above among different segments in Step3_Checkpointing_Neural_Network.py.

In [10]:
# Print the Layers in each segment for the optimal checkpointing configuration
# checkpoints = min_config['optimal_checkpoint_indices']
checkpoints = [1, 3, 5, 7, 10] # Enter manually here for any other configuration corresponding to different n_checkpoints

start_index = 0
for i, checkpoint in enumerate(checkpoints):
    end_index = checkpoint
    print('Segment:',i+1)
    for layer, memory in zip(layer_list[start_index:end_index],memory_list[start_index:end_index]):
        print('Layer =',layer,', Memory =',memory)
    print('\n')
    start_index = end_index

# Last Segment
print('Segment:',len(checkpoints)+1)
for layer, memory in zip(layer_list[start_index:],memory_list[start_index:]):
    print('Layer =',layer,', Memory =',memory)
print('\n')

Segment: 1
Layer =  self.conv1(x) , Memory = 196.9


Segment: 2
Layer =  self.bn1(out) , Memory = 196.3
Layer =  self.relu1(out) , Memory = 196.1


Segment: 3
Layer =  self.mp1(out) , Memory = 147.2
Layer =  self.bn3(self.conv3(self.relu2(self.bn2(self.conv2(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line , Memory = 245.5


Segment: 4
Layer =  self.relu2(out) , Memory = 49.0
Layer =  self.bn5(self.conv5(self.relu3(self.bn4(self.conv4(out))))) + out # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expression is in a single line , Memory = 245.0


Segment: 5
Layer =  self.relu3(out) , Memory = 49.0
Layer =  self.bn7(self.conv7(self.relu4(self.bn6(self.conv6(out))))) + self.downsample20(out) # If we need to add residual after some layers all the layers inbetween are considered as a single layer, hence the expre

In [ ]:
# NOTE: Distribute the layers of the Neural Network as above among different segments in Step3_Checkpointing_Neural_Network.py.